In [1]:
import os
import glob
import json
import pandas as pd
import sqlite3

In [2]:
#  Folder containing JSON files
folder_path = './ipl_json' 

#  Get list of all JSON files
json_files = glob.glob(os.path.join(folder_path, '*.json'))

#  Containers for match and delivery data
all_match_data = []
all_delivery_data = []

# Loop through each file
for file_path in json_files:
    with open(file_path, 'r') as f:
        ipl_data = json.load(f)

    match_id = os.path.basename(file_path).replace('.json', '')
    info = ipl_data.get("info", {})
    
    # Match-level data
    match_data = {
        "match_id": match_id,
        "city": info.get("city", ""),
        "date": info.get("dates", [""])[0],
        "venue": info.get("venue", ""),
        "team1": info.get("teams", ["", ""])[0],
        "team2": info.get("teams", ["", ""])[1],
        "toss_winner": info.get("toss", {}).get("winner", ""),
        "toss_decision": info.get("toss", {}).get("decision", ""),
        "winner": info.get("outcome", {}).get("winner", ""),
        "result_type": list(info.get("outcome", {}).keys())[0] if info.get("outcome") else "",
        "result_margin": list(info.get("outcome", {}).values())[0] if info.get("outcome") else "",
        "player_of_match": info.get("player_of_match", [""])[0]
    }
    all_match_data.append(match_data)

    # Delivery-level data
    innings = ipl_data.get("innings", [])
    for inning_number, inning in enumerate(innings, start=1):
        team = inning.get("team", "")
        overs = inning.get("overs", [])
        
        for over_info in overs:
            over_number = over_info.get("over")
            for delivery in over_info.get("deliveries", []):
                batter = delivery.get("batter")
                bowler = delivery.get("bowler")
                non_striker = delivery.get("non_striker")
                runs = delivery.get("runs", {})
                extras = delivery.get("extras", {})
                total_runs = runs.get("total", 0)
                batter_runs = runs.get("batter", 0)
                extra_runs = runs.get("extras", 0)

                # Handle dismissals
                dismissal_kind = None
                player_dismissed = None
                fielder = None
                if "wickets" in delivery:
                    for w in delivery["wickets"]:
                        dismissal_kind = w.get("kind", "")
                        player_dismissed = w.get("player_out", "")
                        fielder = ", ".join([f.get("name", "") for f in w.get("fielders", [])]) if "fielders" in w else None

                all_delivery_data.append({
                    "match_id": match_id,
                    "inning": inning_number,
                    "batting_team": team,
                    "over": over_number,
                    "batsman": batter,
                    "non_striker": non_striker,
                    "bowler": bowler,
                    "batsman_runs": batter_runs,
                    "extra_runs": extra_runs,
                    "total_runs": total_runs,
                    "dismissal_kind": dismissal_kind,
                    "player_dismissed": player_dismissed,
                    "fielder": fielder
                })


In [3]:
# Create DataFrames
match_df = pd.DataFrame(all_match_data)
deliveries_df = pd.DataFrame(all_delivery_data)

# Preview
print(" Matches loaded:", match_df.shape[0])
print(" Deliveries loaded:", deliveries_df.shape[0])
match_df.head()

 Matches loaded: 1169
 Deliveries loaded: 278205


,match_id,city,date,venue,team1,team2,toss_winner,toss_decision,winner,result_type,result_margin,player_of_match
0,1082591,Hyderabad,2017-04-05,"Rajiv Gandhi International Stadium, Uppal",Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,Sunrisers Hyderabad,by,{'runs': 35},Yuvraj Singh
1,1082592,Pune,2017-04-06,Maharashtra Cricket Association Stadium,Rising Pune Supergiant,Mumbai Indians,Rising Pune Supergiant,field,Rising Pune Supergiant,by,{'wickets': 7},SPD Smith
2,1082593,Rajkot,2017-04-07,Saurashtra Cricket Association Stadium,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,Kolkata Knight Riders,winner,Kolkata Knight Riders,CA Lynn
3,1082594,Indore,2017-04-08,Holkar Cricket Stadium,Kings XI Punjab,Rising Pune Supergiant,Kings XI Punjab,field,Kings XI Punjab,by,{'wickets': 6},GJ Maxwell
4,1082595,Bengaluru,2017-04-08,M.Chinnaswamy Stadium,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat,Royal Challengers Bangalore,winner,Royal Challengers Bangalore,KM Jadhav


In [4]:
match_df.isnull().sum()

match_id           0
city               0
date               0
venue              0
team1              0
team2              0
toss_winner        0
toss_decision      0
winner             0
result_type        0
result_margin      0
player_of_match    0
dtype: int64

In [5]:
deliveries_df

,match_id,inning,batting_team,over,batsman,non_striker,bowler,batsman_runs,extra_runs,total_runs,dismissal_kind,player_dismissed,fielder
0,1082591,1,Sunrisers Hyderabad,0,DA Warner,S Dhawan,TS Mills,0,0,0,None,None,None
1,1082591,1,Sunrisers Hyderabad,0,DA Warner,S Dhawan,TS Mills,0,0,0,None,None,None
2,1082591,1,Sunrisers Hyderabad,0,DA Warner,S Dhawan,TS Mills,4,0,4,None,None,None
3,1082591,1,Sunrisers Hyderabad,0,DA Warner,S Dhawan,TS Mills,0,0,0,None,None,None
4,1082591,1,Sunrisers Hyderabad,0,DA Warner,S Dhawan,TS Mills,0,2,2,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
278200,981019,2,Royal Challengers Bangalore,19,Sachin Baby,CJ Jordan,B Kumar,2,0,2,None,None,None
278201,981019,2,Royal Challengers Bangalore,19,Sachin Baby,CJ Jordan,B Kumar,0,0,0,run out,CJ Jordan,NV Ojha
278202,981019,2,Royal Challengers Bangalore,19,Iqbal Abdulla,Sachin Baby,B Kumar,0,1,1,None,None,None
278203,981019,2,Royal Challengers Bangalore,19,Sachin Baby,Iqbal Abdulla,B Kumar,1,0,1,None,None,None


In [6]:
deliveries_df.isnull().sum()

match_id                 0
inning                   0
batting_team             0
over                     0
batsman                  0
non_striker              0
bowler                   0
batsman_runs             0
extra_runs               0
total_runs               0
dismissal_kind      264382
player_dismissed    264382
fielder             268192
dtype: int64

## All the matches in the data set

In [8]:
print(f'Number of matches Played : {len(match_df)}')

Number of matches Played : 1169


In [9]:
match_df[['match_id','city','date','venue','team1','team2']]

,match_id,city,date,venue,team1,team2
0,1082591,Hyderabad,2017-04-05,"Rajiv Gandhi International Stadium, Uppal",Sunrisers Hyderabad,Royal Challengers Bangalore
1,1082592,Pune,2017-04-06,Maharashtra Cricket Association Stadium,Rising Pune Supergiant,Mumbai Indians
2,1082593,Rajkot,2017-04-07,Saurashtra Cricket Association Stadium,Gujarat Lions,Kolkata Knight Riders
3,1082594,Indore,2017-04-08,Holkar Cricket Stadium,Kings XI Punjab,Rising Pune Supergiant
4,1082595,Bengaluru,2017-04-08,M.Chinnaswamy Stadium,Royal Challengers Bangalore,Delhi Daredevils
...,...,...,...,...,...,...
1164,981011,Raipur,2016-05-22,Shaheed Veer Narayan Singh International Stadium,Delhi Daredevils,Royal Challengers Bangalore
1165,981013,Bangalore,2016-05-24,M Chinnaswamy Stadium,Gujarat Lions,Royal Challengers Bangalore
1166,981015,Delhi,2016-05-25,Feroz Shah Kotla,Sunrisers Hyderabad,Kolkata Knight Riders
1167,981017,Delhi,2016-05-27,Feroz Shah Kotla,Gujarat Lions,Sunrisers Hyderabad


## Maximum Numbers of Matched wins

In [11]:
match_df['winner'].value_counts().reset_index().rename(columns = {
    'index':'Team',
    'winner' :'Wins'
})

,Wins,count
0,Mumbai Indians,151
1,Chennai Super Kings,142
2,Kolkata Knight Riders,135
3,Rajasthan Royals,114
4,Royal Challengers Bangalore,114
5,Sunrisers Hyderabad,93
6,Kings XI Punjab,85
7,Delhi Daredevils,67
8,Delhi Capitals,51
9,Gujarat Titans,37


## Highest Total score in a Match

In [13]:
Highest_score = deliveries_df.groupby(['match_id','batting_team','inning'])['total_runs'].sum().reset_index()
Highest_score.sort_values(by = 'total_runs',ascending = False).reset_index(drop = True).head(1)

,match_id,batting_team,inning,total_runs
0,1426268,Sunrisers Hyderabad,1,287


## Matches Played in Mumbai

In [15]:
match_df[match_df['city'].str.lower() == 'mumbai'][['match_id', 'date', 'team1', 'team2', 'venue']]

,match_id,date,team1,team2,venue
6,1082597,2017-04-09,Mumbai Indians,Kolkata Knight Riders,Wankhede Stadium
9,1082600,2017-04-12,Mumbai Indians,Sunrisers Hyderabad,Wankhede Stadium
15,1082606,2017-04-16,Mumbai Indians,Gujarat Lions,Wankhede Stadium
23,1082614,2017-04-22,Mumbai Indians,Delhi Daredevils,Wankhede Stadium
27,1082618,2017-04-24,Mumbai Indians,Rising Pune Supergiant,Wankhede Stadium
...,...,...,...,...,...
1105,829817,2015-05-19,Chennai Super Kings,Mumbai Indians,Wankhede Stadium
1109,980901,2016-04-09,Mumbai Indians,Rising Pune Supergiants,Wankhede Stadium
1117,980917,2016-04-16,Mumbai Indians,Gujarat Lions,Wankhede Stadium
1122,980927,2016-04-20,Mumbai Indians,Royal Challengers Bangalore,Wankhede Stadium


## Who Scored the Most Runs Across All Matches?

In [17]:
deliveries_df.groupby('batsman')['batsman_runs'].sum().sort_values(ascending = False).reset_index().rename(columns={
    'batsman': 'Player',
    'batsman_runs':'Total runs'
}).head(10)

,Player,Total runs
0,V Kohli,8671
1,RG Sharma,7048
2,S Dhawan,6769
3,DA Warner,6567
4,SK Raina,5536
5,MS Dhoni,5439
6,KL Rahul,5235
7,AB de Villiers,5181
8,AM Rahane,5032
9,CH Gayle,4997


## Which Bowler Took the Most Wickets?

In [19]:
wickets_df = deliveries_df[deliveries_df['dismissal_kind'].notnull()]
wickets_df.groupby('bowler')['player_dismissed'].count().sort_values(ascending=False).reset_index().rename(columns={
    'bowler': 'Bowler',
    'player_dismissed': 'Wickets'
}).head(10)


,Bowler,Wickets
0,YS Chahal,229
1,B Kumar,213
2,SP Narine,212
3,DJ Bravo,207
4,R Ashwin,205
5,JJ Bumrah,203
6,PP Chawla,201
7,SL Malinga,188
8,A Mishra,183
9,RA Jadeja,179


## Virat Kohli's Batting Stats

In [21]:
kohli_df = deliveries_df[deliveries_df['batsman'] == 'V Kohli']

matches = kohli_df['match_id'].nunique()
total_runs = kohli_df['batsman_runs'].sum()
balls = kohli_df.shape[0]
strike_rate = round(total_runs / balls * 100, 2)

print("Matches Played:", matches)
print("Total Runs:", total_runs)
print("Balls Faced:", balls)
print("Strike Rate:", strike_rate)



Matches Played: 259
Total Runs: 8671
Balls Faced: 6702
Strike Rate: 129.38


## Best Bowling Figures in a Single Match

In [23]:
wickets_df.groupby(['match_id', 'bowler'])['player_dismissed'].count().reset_index().sort_values(by='player_dismissed', ascending=False).head(1)


,match_id,bowler,player_dismissed
4430,336005,Sohail Tanvir,6


## Average First Innings Score

In [25]:
first_innings_scores = deliveries_df[deliveries_df['inning'] == 1].groupby('match_id')['total_runs'].sum()
average_score = round(first_innings_scores.mean(), 2)
print("Average First Innings Score:", average_score)

Average First Innings Score: 167.02


## Venue with Highest Average Score

In [27]:
match_totals = deliveries_df.groupby('match_id')['total_runs'].sum().reset_index()
merged = pd.merge(match_totals, match_df[['match_id', 'venue']], on='match_id')
merged.groupby('venue')['total_runs'].mean().sort_values(ascending=False).head(5)


venue
Maharaja Yadavindra Singh International Cricket Stadium, New Chandigarh    436.000000
Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium, Visakhapatnam         387.250000
Arun Jaitley Stadium, Delhi                                                384.130435
Eden Gardens, Kolkata                                                      372.434783
Punjab Cricket Association IS Bindra Stadium, Mohali, Chandigarh           371.200000
Name: total_runs, dtype: float64

## All Century scored

In [29]:
centuries = deliveries_df.groupby(['match_id', 'batsman'])['batsman_runs'].sum().reset_index()
centuries = centuries[centuries['batsman_runs'] >= 100].sort_values(by='batsman_runs', ascending=False)
centuries


,match_id,batsman,batsman_runs
14323,598027,CH Gayle,175
9023,335982,BB McCullum,158
8317,1473464,Abhishek Sharma,141
5491,1304112,Q de Kock,140
16549,829795,AB de Villiers,133
...,...,...,...
5908,1359493,HC Brook,100
6641,1359539,V Kohli,100
6693,1359543,C Green,100
7067,1422137,JC Buttler,100


## The most successful chase target?

In [31]:
# First innings scores
first_innings = deliveries_df[deliveries_df['inning'] == 1].groupby('match_id')['total_runs'].sum().reset_index()
first_innings.columns = ['match_id', 'target']
first_innings['target'] += 1
# Second innings scores
second_innings = deliveries_df[deliveries_df['inning'] == 2].groupby('match_id')['total_runs'].sum().reset_index()
second_innings.columns = ['match_id', 'chased']

# Merge and filter successful chases
merged = pd.merge(first_innings, second_innings, on='match_id')
successful_chases = merged[merged['chased'] >= merged['target']]
successful_chases = successful_chases.sort_values(by='target', ascending=False)
successful_chases.head(1)

,match_id,target,chased
487,1426280,262,262


## Which team has the best powerplay performance?

In [33]:
powerplay_df = deliveries_df[deliveries_df['over'] <= 5]
powerplay_scores = powerplay_df.groupby(['match_id', 'batting_team'])['total_runs'].sum().reset_index()

# Average per team
team_powerplay_avg = powerplay_scores.groupby('batting_team')['total_runs'].mean().sort_values(ascending=False)
team_powerplay_avg.head(5)


batting_team
Royal Challengers Bengaluru    58.600000
Punjab Kings                   52.918919
Delhi Capitals                 51.980952
Gujarat Lions                  51.966667
Gujarat Titans                 50.233333
Name: total_runs, dtype: float64

## The scorecard for match between CSK and MI

In [35]:
csk_vs_mi = match_df[
    ((match_df['team1'] == 'Chennai Super Kings') & (match_df['team2'] == 'Mumbai Indians')) |
    ((match_df['team2'] == 'Chennai Super Kings') & (match_df['team1'] == 'Mumbai Indians'))
]

#  Count number of wins per team in these matches
head_to_head = csk_vs_mi['winner'].value_counts().reset_index()
head_to_head.columns = ['Team', 'Wins_vs_each_other']

print("Head-to-Head (CSK vs MI):")
print(head_to_head)



Head-to-Head (CSK vs MI):
                  Team  Wins_vs_each_other
0       Mumbai Indians                  21
1  Chennai Super Kings                  18


## Sixes in the finals

In [37]:
final_match_id = match_df.sort_values('date').iloc[-1]['match_id']

sixes = deliveries_df[(deliveries_df['match_id'] == final_match_id) & (deliveries_df['batsman_runs'] == 6)]
print("Total Sixes in Final:", sixes.shape[0])


Total Sixes in Final: 23


## Partnerships over 100 runs

In [39]:
partnerships = deliveries_df.groupby(['match_id', 'inning', 'batsman', 'non_striker'])['total_runs'].sum().reset_index()
partnerships['pair'] = partnerships.apply(lambda row: tuple(sorted([row['batsman'], row['non_striker']])), axis=1)
partnerships = partnerships.groupby(['match_id', 'inning', 'pair'])['total_runs'].sum().reset_index()
big_partnerships = partnerships[partnerships['total_runs'] >= 100].sort_values(by='total_runs', ascending=False)
big_partnerships


,match_id,inning,pair,total_runs
15469,980987,1,"(AB de Villiers, V Kohli)",229
14734,829795,1,"(AB de Villiers, V Kohli)",215
6858,1426297,1,"(B Sai Sudharsan, Shubman Gill)",210
4916,1304112,1,"(KL Rahul, Q de Kock)",210
11219,501260,1,"(AC Gilchrist, SE Marsh)",206
...,...,...,...,...
5209,1359487,2,"(N Rana, VR Iyer)",100
3030,1216540,1,"(AB de Villiers, V Kohli)",100
2800,1216520,2,"(CH Gayle, Mandeep Singh)",100
14238,829717,2,"(Harbhajan Singh, J Suchith)",100
